In [10]:
!pip freeze | grep scikit-learn

scikit-learn==1.0.2


In [11]:
import pickle
import pandas as pd

In [12]:
# declaring year and month
year = 2021
month = 2

In [13]:
with open('model.bin', 'rb') as f_in:
    dv, lr = pickle.load(f_in)

In [14]:
categorical = ['PUlocationID', 'DOlocationID']

def read_data(filename):
    df = pd.read_parquet(filename)
    
    df['duration'] = df.dropOff_datetime - df.pickup_datetime
    df['duration'] = df.duration.dt.total_seconds() / 60

    df = df[(df.duration >= 1) & (df.duration <= 60)].copy()

    df[categorical] = df[categorical].fillna(-1).astype('int').astype('str')
    
    return df

In [15]:
df = read_data(f'https://nyc-tlc.s3.amazonaws.com/trip+data/fhv_tripdata_{year:04d}-{month:02d}.parquet')

In [16]:
dicts = df[categorical].to_dict(orient='records')
X_val = dv.transform(dicts)
y_pred = lr.predict(X_val)

In [17]:
print(y_pred.mean())

16.191691679979066


In [18]:
# Get unique ride it and creating a new dataframe
df['ride_id'] = f'{year:04d}/{month:02d}_' + df.index.astype('str')
df['predictions'] = y_pred

df_result = df[['ride_id', 'predictions']].copy()

In [20]:
# writing results to parquet
output_file = f"output/{year:04d}-{month:02d}-results.parquet"
df_result.to_parquet(output_file, engine='pyarrow', compression=None, index=False)

In [23]:
import pyarrow.parquet as pq
pq.ParquetFile(output_file).metadata

  created_by: parquet-cpp-arrow version 8.0.0
  num_columns: 2
  num_rows: 990113
  num_row_groups: 1
  format_version: 1.0
  serialized_size: 1561